# Single-Cell Report: Filtering and QC

In [ ]:
import os
print("Files in use:")
print(f"FILE1='{os.path.abspath(FILE1)}'")
print(f"FILE2='{os.path.abspath(FILE2)}'")

In [ ]:
# Import packages
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pandas as pd
from collections import Counter

#### Plotting settings and functions

In [ ]:
# plot settings
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)
hist_dims = (10,3)

In [ ]:
def plotSummaryHist(adata, nCellsPerGene, figsize=(10,3), draw_thresholds=False, log_x=[False]*4, log_y=[False]*4):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=figsize, dpi=150, sharey=False)

    sns.distplot( adata.obs['n_genes'], ax=ax1, norm_hist=True, bins='fd')
    ax1.title.set_text('Genes expressed per cell')

    sns.distplot( adata.obs['n_counts'], ax=ax2, norm_hist=True, bins='fd')
    ax2.title.set_text('Counts per cell')

    if 'percent_mito' in adata.obs.keys():
        sns.distplot( adata.obs['percent_mito'], ax=ax3, norm_hist=True, bins='fd')
        ax3.title.set_text('Mitochondrial read fraction per cell')
        if(draw_thresholds and not adata.uns['sc']['scanpy']['filter']['cellFilterMaxPercentMito'] == -1):
            ax3.axvline(x=adata.uns['sc']['scanpy']['filter']['cellFilterMaxPercentMito'], ymin=0,ymax=1, color='red')
    else:
        warnings.warn("Percentage of mitochondrial genes expressed in cells not calculated")
        
    nc_cnt = Counter(nCellsPerGene)
    nc_cnt_df = pd.DataFrame( [list(nc_cnt.keys()), list(nc_cnt.values())] ).T.sort_values(by=0)
    ax4.bar(nc_cnt_df[0], nc_cnt_df[1] )
    ax4.margins(x=0.01)
    ax4.title.set_text('Cells expressing each gene')
    
    # set axes to log-scale:
    for i,ax in enumerate((ax1,ax2,ax3,ax4)):
        ax.tick_params(axis='y', which='major', labelsize=8)
        if log_x[i]:
            ax.set_xscale('log')
        if log_y[i]:
            ax.set_yscale('log')

    if(draw_thresholds):
        if not adata.uns['sc']['scanpy']['filter']['cellFilterMinNGenes'] == -1:
            ax1.axvline(x=adata.uns['sc']['scanpy']['filter']['cellFilterMinNGenes'], ymin=0,ymax=1, color='red')
        if not adata.uns['sc']['scanpy']['filter']['cellFilterMaxNGenes'] == -1:
            ax1.axvline(x=adata.uns['sc']['scanpy']['filter']['cellFilterMaxNGenes'], ymin=0,ymax=1, color='red')
        if not adata.uns['sc']['scanpy']['filter']['cellFilterMinNCounts'] == -1:
            ax2.axvline(x=adata.uns['sc']['scanpy']['filter']['cellFilterMinNCounts'], ymin=0,ymax=1, color='red')
        if not adata.uns['sc']['scanpy']['filter']['cellFilterMaxNCounts'] == -1:
            ax2.axvline(x=adata.uns['sc']['scanpy']['filter']['cellFilterMaxNCounts'], ymin=0,ymax=1, color='red')
        if not adata.uns['sc']['scanpy']['filter']['geneFilterMinNCells'] == -1:
            ax4.axvline(x=adata.uns['sc']['scanpy']['filter']['geneFilterMinNCells']-0.5, ymin=0,ymax=1, color='red')

    
    fig.text(0.00, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')
    fig.tight_layout()

In [ ]:
def plotSingleDiagnosticHist(x, nbins='fd', filter_thr=None, xlab="", log_x=False, log_y=False, kde=True):
    
    fig, ax = plt.subplots(1, 1, figsize=hist_dims, dpi=150)
    
    sns.distplot(x, ax=ax, norm_hist=True, kde=kde, bins=nbins)
    ax.set_xlabel("")    
            
    ax.margins(x=0.01)
    if type(filter_thr)==list:
        if filter_thr[1] > ax.get_xlim()[1]:
            ax.set_xlim(right=filter_thr[1])
        if filter_thr[0] < ax.get_xlim()[0]:
            ax.set_xlim(left=filter_thr[0])        
        for i,x in enumerate(filter_thr):
            if not x == -1:
                ax.axvline(x=x, ymin=0,ymax=1, color='red')
    elif filter_thr != -1:
        ax.axvline(x=filter_thr, ymin=0,ymax=1, color='red')
    
    if log_x:
        ax.set_xscale('log')
    if log_y:
        ax.set_yscale('log')
    
    fig.text(0.00, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')
    fig.text(0.5, 0.0, xlab, ha='center', va='center', size='x-large')
    
    fig.tight_layout()

In [ ]:
def plotSingleDiagnosticBar(x, filter_thr=None, xlab="", log_x=False, log_y=False):

    fig, ax = plt.subplots(1, 1, figsize=hist_dims, dpi=150)

    nc_cnt = Counter(x+0.1)
    nc_cnt_df = pd.DataFrame( [list(nc_cnt.keys()), list(nc_cnt.values())] ).T.sort_values(by=0)
    ax.bar(nc_cnt_df[0], nc_cnt_df[1])
    ax.set_xlabel("")

    #nc_cnt_df_sub = nc_cnt_df[ nc_cnt_df[0]<20 ]
    #for i,r in nc_cnt_df_sub.iterrows():
    #    plt.text(r[0], r[1], r[0], fontsize='small', ha='center') # display the x value on each bar

    ax.margins(x=0.01)
    if filter_thr != -1:
        ax.axvline(x=filter_thr, ymin=0,ymax=1, color='red')
    
    if log_x:
        ax.set_xscale('log')
    if log_y:
        ax.set_yscale('log')

    fig.text(0.00, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')
    fig.text(0.5, 0.0, xlab, ha='center', va='center', size='x-large')
    
    fig.tight_layout()

## Read Data

We read in the data pre-filtered, and post-filtered, with the filtering parameters specified in the nextflow config file applied.

In [ ]:
adata_pre = sc.read_h5ad(filename=FILE1)
adata_post = sc.read_h5ad(filename=FILE2)

In [ ]:
sum(np.isnan(adata_pre.obs["percent_mito"]))

In [ ]:
adata_pre.obs = adata_pre.obs.fillna(
    value={
        "percent_mito":0
    }
)

---
# Prefilter diagnostics

#### Basic Gene-Level Summary Statistics

In [ ]:
n_counts_per_gene = np.sum(adata_pre.X, axis=0)
n_cells_per_gene = np.sum(adata_pre.X>0, axis=0)
print("Number of counts (in the dataset units) per gene:", n_counts_per_gene.min(), " - " ,n_counts_per_gene.max())
print("Number of cells in which each gene is detected:", n_cells_per_gene.min(), " - " ,n_cells_per_gene.max())
nCellsPerGene_pre = pd.Series( n_cells_per_gene.tolist()[0], index=adata_pre.var_names )

## Diagnostic Plots (pre-filtering)

#### Highly expressed genes

Show the genes with the highest fraction of counts assigned to that gene over all the cells.

In [ ]:
sc.pl.highest_expr_genes(adata_pre, n_top=20)

#### Summary histograms showing the distributions of number of genes, number of counts, and fraction of mitochondrial genes

Please note that the histogram for the number of cells expressing each gene has a log-scale for both the x- and y-axis.

In [ ]:
plotSummaryHist(adata_pre, nCellsPerGene_pre, 
                draw_thresholds=True,
                log_x=[False]*3+[True],
                log_y=[False]*3+[True])

#### Summary violin plots showing the distributions of number of genes, number of counts, and fraction of mitochondrial genes

In [ ]:
metrics = ['n_genes', 'n_counts', 'percent_mito']
sc.pl.violin(adata_pre, np.array(metrics)[np.isin(metrics,adata_pre.obs.keys())],
    jitter=0.4, multi_panel=True )

#### Scatter plot showing the number of genes vs number of counts, colored by mitochondrial read fraction

In [ ]:
if 'percent_mito' in adata_pre.obs.keys():
    sc.pl.scatter(adata_pre, x='n_counts', y='n_genes', color='percent_mito', title="Fraction of mitochondrial reads per cell")
else:
    sc.pl.scatter(adata_pre, x='n_counts', y='n_genes')
    warnings.warn("Percentage of mitochondrial genes expressed in cells not calculated")

---
## Setting the filters

In [ ]:
print(adata_pre.uns['sc']['scanpy']['filter'])

### Filter 1: Number of genes expressed per cell

**Cell-level filtering**

To determine whether the thresholds are set correctly, it's useful to look at a histogram showing the distribution of the number of genes expressed for each cell. Here, the entire range of the data (equivalent to the summary plot above) is shown on a wide x-axis. The current thresholds are show in red lines (it's possible that one or both thresholds are outside the plot limits; in this case the threshold should likely be raised/lowered to better fit the data).

In [ ]:
thr = [ adata_pre.uns['sc']['scanpy']['filter']['cellFilterMinNGenes'],
        adata_pre.uns['sc']['scanpy']['filter']['cellFilterMaxNGenes'] ]
plotSingleDiagnosticHist(adata_pre.obs['n_genes'],
                         xlab='Number of genes expressed per cell',
                         filter_thr=thr)
print("Number of genes expressed per cell threshold is set to {}-{}".format(thr[0],thr[1]))
print("Unfiltered data range is {}-{}.".format(adata_pre.obs['n_genes'].min(),adata_pre.obs['n_genes'].max() ))

### Filter 2: Number of counts expressed per cell

**Cell-level filtering**

To determine whether the thresholds are set correctly, it's useful to look at a histogram showing the distribution of the number of genes expressed for each cell. Here, the entire range of the data (equivalent to the summary plot above) is shown on a wide x-axis. The current thresholds are show in red lines (it's possible that one or both thresholds are outside the plot limits; in this case the threshold should likely be raised/lowered to better fit the data).

In [ ]:
thr = [ adata_pre.uns['sc']['scanpy']['filter']['cellFilterMinNCounts'],
        adata_pre.uns['sc']['scanpy']['filter']['cellFilterMaxNCounts'] ]
plotSingleDiagnosticHist(adata_pre.obs['n_counts'],
                         xlab='Number of counts expressed per cell',
                         filter_thr=thr)
print("Number of counts expressed per cell threshold is set to {}-{}".format(thr[0],thr[1]))
print("Unfiltered data range is {}-{}.".format(adata_pre.obs['n_counts'].min(),adata_pre.obs['n_counts'].max() ))

### Filter 3: Fraction of mitochondrial reads per cell

**Cell-level filtering**

The same approach is used to examine the fraction of mitochondrial reads per cell. In this case, there is only an upper threshold on mitochondrial read fraction (cells with a mitochondrial read fraction above this threshold will be discarded).

In [ ]:
if 'percent_mito' in adata_pre.obs.keys():
    thr = adata_pre.uns['sc']['scanpy']['filter']['cellFilterMaxPercentMito']
    plotSingleDiagnosticHist(adata_pre.obs['percent_mito'],
                         xlab='Fraction of mitochondrial reads per cell',
                         filter_thr=thr)
    print("Fraction of mitochondrial reads per cell threshold limit is set to {}".format(thr))
    print("Unfiltered data range is {}-{}.".format(adata_pre.obs['percent_mito'].min(),adata_pre.obs['percent_mito'].max() ))
else:
    warnings.warn("Percentage of mitochondrial genes expressed in cells not calculated")

### Filter 4: Number of cells expressing each gene

**Gene-level filtering**

The final filter is applied on a gene-level: For each gene, we count the number of cells in which it is expressed. The distribution here is usually highly skewed, such that there are a large number of genes expressed in only a few cells (with many expressed in 0 or 1 cells). The threshold here can be conservatively set low: requiring a minimum of **3** cells expressing any given gene is a reasonable start, but this will be highly dependent on the data, particularly the total number of cells in the experiment.

The same approach is used as above, with the x and y axes displayed in log scale (this causes the bars at the lower end to be very wide, but this is the region of interest anyway). In this case, there is only an upper threshold on number of cells expressing each gene.

In [ ]:
thr = adata_pre.uns['sc']['scanpy']['filter']['geneFilterMinNCells']
plotSingleDiagnosticBar(nCellsPerGene_pre,
                         xlab='Number of cells expressing each gene',
                         filter_thr=thr-0.5, log_x=True, log_y=True)

print("Keeping genes present in at least {} cells".format(thr))
print("Unfiltered data range is {}-{}.".format(nCellsPerGene_pre.min(),nCellsPerGene_pre.max() ))

---
## Diagnostic Plots (post-filtering)

Here, we show repeat the summary plots shown above, this time showing the data **after filting**.

In [ ]:
# re-calculate number of cells per gene for post-filter:
nCellsPerGene_post = pd.Series( np.sum(adata_post.X>0, axis=0).tolist()[0], index=adata_post.var_names )

In [ ]:
plotSummaryHist(adata_post, nCellsPerGene_post,
                draw_thresholds=False,
                log_x=[False]*3+[True],
                log_y=[False]*3+[True])

#### Violin plots of number of genes, number of counts, and percent of mitochondrial genes

In [ ]:
metrics = ['n_genes', 'n_counts', 'percent_mito']
sc.pl.violin(adata_post, np.array(['n_genes', 'n_counts', 'percent_mito'])[np.isin(metrics,adata_post.obs.keys())],
    jitter=0.4, multi_panel=True )

#### Scatter plot number of genes vs number of counts

In [ ]:
if 'percent_mito' in adata_post.obs.keys():
    sc.pl.scatter(adata_post, x='n_counts', y='n_genes', color='percent_mito', title="Fraction of mitochondrial reads per cell")
else:
    sc.pl.scatter(adata_post, x='n_counts', y='n_genes')
    warnings.warn("Percentage of mitochondrial genes expressed in cells not calculated")

---
## Summary

In [ ]:
print(f"Pre-filtered data: {FILE1}\n\t{adata_pre._n_obs} cells\n\t{adata_pre._n_vars} genes")
#
print(f"Filters applied:\n\tNumber of genes expressed per cell:")
thr = [ adata_pre.uns['sc']['scanpy']['filter']['cellFilterMinNGenes'], adata_pre.uns['sc']['scanpy']['filter']['cellFilterMaxNGenes'] ]
print("\t\tUnfiltered data range is {}-{}.".format(adata_pre.obs['n_genes'].min(),adata_pre.obs['n_genes'].max() ))
print("\t\tthreshold is set to {}-{}".format(thr[0],thr[1]))
print("\t\tFiltered data range is {}-{}.".format(adata_post.obs['n_genes'].min(),adata_post.obs['n_genes'].max() ))
#
if 'percent_mito' in adata_pre.obs.keys():
    print(f"\tFraction of mitochondrial reads per cell:")
    thr = adata_pre.uns['sc']['scanpy']['filter']['cellFilterMaxPercentMito']
    print(f"\t\tUnfiltered data range is {adata_pre.obs['percent_mito'].min()}-{adata_pre.obs['percent_mito'].max()}.")
    print(f"\t\tthreshold limit is set to {thr}")
    print(f"\t\tFiltered data range is {adata_post.obs['percent_mito'].min()}-{adata_post.obs['percent_mito'].max()}.")
#
print(f"\tNumber of cells expressing each gene:")
thr = adata_pre.uns['sc']['scanpy']['filter']['geneFilterMinNCells']
print(f"\t\tUnfiltered data range is {nCellsPerGene_pre.min()}-{nCellsPerGene_pre.max()}.")
print(f"\t\tthreshold is set to keep genes present in at least {thr} cells")
print(f"\t\tFiltered data range is {nCellsPerGene_post.min()}-{nCellsPerGene_post.max()}.")
#
print(f"Post-filtered data: {FILE2}")
print(f"\t{adata_post._n_obs} cells (removed {adata_pre._n_obs - adata_post._n_obs})")
print(f"\t{adata_post._n_vars} genes (removed {adata_pre._n_vars - adata_post._n_vars})")